# Trajectory Identification (TI)

> DifferenceNet as ann alternative to Siamese Net for TI

<p align="left">
  <img src="https://github.com/FKTechLab/DifferenceNet/blob/main/ti/images/traj_half.png" width="30%" />
  <img src="https://github.com/FKTechLab/DifferenceNet/blob/main/ti/images/traj_full.png" width="29.3%" />
</p>

In [15]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *
import warnings
warnings.filterwarnings("ignore")

DSN is a special type to Siamese Net where the difference of positive and anchor and negative and anchor is used to create a single input vector for the geospatial and temporal model.

## Install

`pip install git+https://github.com/arunabha4k/DSN.git@master`

## How to use

usage: train.py [-h] [-m {SN,DN}] [-e EPOCH] [-p PRINT]

optional arguments:

    -h, --help            show this help message and exit

    -m {SN,DN}, --model {SN,DN}
    Choose model, SN : Siamese Net, DN : Difference Net

    -e EPOCH, --epoch EPOCH
    Number of Epochs

    -p PRINT, --print PRINT
    Print at every p step, p must not be greater than e

### Transformer

In [2]:
import pandas as pd
from ti.prep import Transformer

X = pd.read_csv('../data/sim/0.csv')
transformer = Transformer()
x1, x2 = transformer.transform(X)
print(len(x1), len(x2))
print(len(x1[0]), len(x2[0]))

9 9
4 4


### Model

#### Difference Net
This module helps create the differential features using (simple sub, abs diff or squared diff) for:
- (pos, anchor) or
- (neg, anchor)

In [3]:
from torch.utils.data import DataLoader

from ti.dataloader import DatasetTraj, zero_padding
from ti.prep import Transformer
from ti.model import Difference

# Data generator
params = {
    'batch_size': 3,
    'shuffle': True,
    'collate_fn': zero_padding
}
train_g = DataLoader(DatasetTraj(range(0,4), [[0,1], [2,3], [1,3], [0, 1], [2,4], [6,0], [5,1], [9,3], [11,7], \
                                              [8, 1], [10,4], [4,10]], mode='real'), **params)
diff_s = Difference(mode='simple')
diff_a = Difference(mode='abs')
diff_sq = Difference(mode='square')
for x1, x2, y, x_seq_lens, max_seq_len in train_g:
    print(x1.shape)
    print('simple')
    print(diff_s(x1, x2[0])[0])
    print('abs')
    print(diff_a(x1, x2[0])[0])
    print('square')
    print(diff_sq(x1, x2[0])[0])

torch.Size([3, 53, 4])
simple
tensor([[-5.2571e-05, -4.2319e-06,  3.0102e-01,  5.0041e-02],
        [ 9.0423e-04,  1.5163e-04,  3.0767e-01,  5.0095e-02],
        [ 2.4011e-03,  1.8251e-04,  3.1234e-01,  5.0107e-02],
        [ 3.3363e-03,  4.9114e-04,  3.1432e-01,  5.0106e-02],
        [ 4.6635e-03,  8.7696e-04,  3.1707e-01,  5.0097e-02],
        [ 6.5585e-03,  1.8214e-03,  3.2174e-01,  5.0065e-02],
        [ 6.5431e-03,  1.8939e-03,  3.2196e-01,  5.0063e-02],
        [ 8.5184e-03,  2.4773e-03,  3.2642e-01,  5.0012e-02],
        [ 1.0648e-02,  3.2597e-03,  3.3106e-01,  4.9937e-02],
        [ 1.2244e-02,  4.2566e-03,  3.3519e-01,  4.9852e-02],
        [ 1.2432e-02,  4.4202e-03,  3.3577e-01,  4.9838e-02],
        [ 1.3747e-02,  5.7766e-03,  3.4039e-01,  4.9720e-02],
        [ 1.3716e-02,  5.6979e-03,  3.4245e-01,  4.9661e-02],
        [ 1.3716e-02,  5.6964e-03,  3.5223e-01,  4.9321e-02],
        [ 1.4481e-02,  6.6347e-03,  3.5690e-01,  4.9125e-02],
        [ 1.5126e-02,  8.0822e-03,  3.61

#### Trajectory Identtification: Difference Net

![alt text](https://github.com/FKTechLab/DifferenceNet/blob/main/ti/images/TrajectoryDN.jpeg)

In [4]:
from ti.model import TrajectoryDN
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
diff_net = Difference(mode='simple')
net = TrajectoryDN(diff_net, n_features=len(transformer.features_traj)*2) # 2x for org and dest 
# Forward Pass
count = 0
for x1, x2, y, x_seq_lens, max_seq_len in train_g:
    x1, y, x_seq_lens = torch.Tensor(x1).to(device), torch.Tensor(y).to(device), torch.Tensor(x_seq_lens).to(device)
    org = x2[0]
    dst = x2[1]
    org = torch.Tensor(org).to(device)
    dst = torch.Tensor(dst).to(device)
    x2 = [org, dst]
    print('Batch')
    print(x1.shape)
    print(x2[0].shape, x2[0].shape)
    print(y)
    print(x_seq_lens)
    print(max_seq_len)
    print(net(x1, x2, x_seq_lens))
    if count >= 0:
        break
    count+=1

Batch
torch.Size([3, 38, 4])
torch.Size([3, 38, 4]) torch.Size([3, 38, 4])
tensor([1., 0., 1.])
tensor([38., 20., 19.])
38
tensor([[0.4933],
        [0.4934],
        [0.4934]], grad_fn=<SigmoidBackward0>)


#### Trajectory Identtification: Siamese Net

![alt text](https://github.com/FKTechLab/DifferenceNet/blob/main/ti/images/TrajectorySN.jpeg)

In [5]:
import torch
from ti.model import TrajectorySN, ContrastiveLoss

net = TrajectorySN(n_features=len(transformer.features_traj)) 
criterion = ContrastiveLoss()
# Forward Pass
count = 0
for x1, x2, y, x_seq_lens, max_seq_len in train_g:
    print('Batch')
    print(x1.shape)
    print(x2[0].shape, x2[0].shape)
    print(y)
    y = torch.Tensor(y)
    print(x_seq_lens)
    print(max_seq_len)
    x1, x2 = net(x1, x2, x_seq_lens)
    print(x1.shape, x2.shape)
    print(criterion(x1, x2, y).item())
    if count >= 0:
        break
    count+=1

Batch
torch.Size([3, 40, 4])
torch.Size([3, 40, 4]) torch.Size([3, 40, 4])
[0, 0, 0]
[21, 40, 35]
40
torch.Size([3, 16]) torch.Size([3, 16])
0.2231968492269516
